# IE402 Assignment 2
## By: 201801015
### 11-10-2020

In [ ]:
import numpy as np
from collections import Counter

# inputs
n, m = input("Enter no. of Rows and Columns seperated by a space:").split(" ")
n, m = int(n), int(m)
# cost
mat = np.zeros((n,m))
# supply
ai = np.zeros(n)
# demand
bi = np.zeros(m)
for i in range(n):
    mat[i] = input("Enter " + str(m) + " values for Row " +str(i+1) +" seperated by a space:").split(" ")
ai = input("Enter " + str(n) + " Ai values seperated by a space:").split(" ")
bi = input("Enter " + str(m) + " Bi values seperated by a space:").split(" ")

# VAN method
# Only solves balanced problem
#assert sum(ai) == sum(bi)

s = np.copy(ai)
d = np.copy(bi)
C = np.copy(mat)

n, m = C.shape

# Finding initial solution
X = np.zeros((n, m))
indices = [(i, j) for i in range(n) for j in range(m)]
xs = sorted(zip(indices, C.flatten()), key=lambda t: t[1])

# Iterating C elements in increasing order
for (i, j), _ in xs:
    if d[j] == 0:
        continue
    else:
        # Reserving supplies in a greedy way
        remains = int(s[i]) - int(d[j]) if s[i] >= d[j] else 0
        grabbed = int(s[i]) - remains
        X[i, j] = grabbed
        s[i] = remains
        d[j] = int(d[j]) - grabbed

# Finding optimal solution
while True:
    u = np.array([np.nan]*n)
    v = np.array([np.nan]*m)
    S = np.zeros((n, m))

    _x, _y = np.where(X > 0)
    nonzero = list(zip(_x, _y))
    f = nonzero[0][0]
    u[f] = 0

    # Finding u, v potentials
    while any(np.isnan(u)) or any(np.isnan(v)):
        for i, j in nonzero:
            if np.isnan(u[i]) and not np.isnan(v[j]):
                u[i] = C[i, j] - v[j]
            elif not np.isnan(u[i]) and np.isnan(v[j]):
                v[j] = C[i, j] - u[i]
            else:
                continue

    # Finding S-matrix
    for i in range(n):
        for j in range(m):
            S[i, j] = C[i, j] - u[i] - v[j]

    # Stop condition
    s = np.min(S)
    if s >= 0:
        break

    i, j = np.argwhere(S == s)[0]
    start = (i, j)

    # Finding cycle elements
    T = np.copy(X)
    T[start] = 1  
    while True:
        _xs, _ys = np.nonzero(T)
        xcount, ycount = Counter(_xs), Counter(_ys)

        for x, count in xcount.items():
            if count <= 1:
                T[x,:] = 0
        for y, count in ycount.items():
            if count <= 1: 
                T[:,y] = 0

        if all(x > 1 for x in xcount.values()) \
                and all(y > 1 for y in ycount.values()):
            break

    # Finding cycle chain order
    dist = lambda t: abs(t[0]-t[1]) + abs(t[2]-t[3])
    fringe = set(tuple(p) for p in np.argwhere(T > 0))

    size = len(fringe)

    path = [start]
    while len(path) < size:
        last = path[-1]
        if last in fringe:
            fringe.remove(last)
        next = min(fringe, key=lambda t: dist(last, (t[0], t[1])))
        path.append(next)

    # Improving solution on cycle elements
    neg = path[1::2]
    pos = path[::2]
    q = min(X[zip(*neg)])
    X[zip(*neg)] -= q
    X[zip(*pos)] += q
    
routes, z = X, np.sum(X*C)

print(routes)
print(z)

# outputs
print('Initial BFS: ')
print('Cost: ')
print('Optimal Solution: ')
print('Optimal Cost: ')

Enter no. of Rows and Columns seperated by a space:3 4
Enter 4 values for Row 1 seperated by a space:19 30 50 10
Enter 4 values for Row 2 seperated by a space:70 30 40 60
Enter 4 values for Row 3 seperated by a space:40 8 70 20
Enter 3 Ai values seperated by a space:7 9 18
Enter 4 Bi values seperated by a space:5 8 7 14
